In [ ]:
pip install MIDASpy

     |████████████████████████████████| 1.1 MB 13.3 MB/s 
  Created wheel for MIDASpy: filename=MIDASpy-1.2.1-py3-none-any.whl size=19841 sha256=3229af64d52557caf3658354962aeef7ab5dc7392ac0486a4b3d16d042739e7d
  Stored in directory: /root/.cache/pip/wheels/ba/df/6b/cffe0cf4fa08ef29a00e0d0e0863673222579adf2f36e063bf
Successfully built MIDASpy


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
#import MIDASpy as md

In [ ]:
!wget https://github.com/eddie-uu/pattern_recognition/raw/main/databases/nasa_filtered.csv

--2022-01-28 13:08:17--  https://github.com/eddie-uu/pattern_recognition/raw/main/databases/nasa_filtered.csv
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/eddie-uu/pattern_recognition/main/databases/nasa_filtered.csv [following]
--2022-01-28 13:08:17--  https://raw.githubusercontent.com/eddie-uu/pattern_recognition/main/databases/nasa_filtered.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5768919 (5.5M) [text/plain]
Saving to: ‘nasa_filtered.csv’

nasa_filtered.csv   100%[===================>]   5.50M  --.-KB/s    in 0.09s   

2022-01-28 13:08:18 (60.3 MB/s) - ‘nasa_filtered.csv’ saved 

In [ ]:
data = pd.read_csv("nasa_filtered.csv", error_bad_lines=False)

In [ ]:
# randomly remove some data
def random_remove(df, rate):
  df = pd.DataFrame(df)
  rate = rate+df.isnull().sum().sum()/(df.shape[0]*df.shape[1])
  dataset = df.reset_index()
  melt_one = pd.melt(dataset, id_vars = ['index'])
  sampled = melt_one.sample(frac = rate, random_state=1).reset_index(drop = True)
  dataset = sampled.pivot(index = 'index', columns = 'variable', values= 'value')
  dataset = np.asarray(dataset).astype('float32')
  return dataset

# impute mean value into dataset
def fill_mean(dataset):
  dataset = pd.DataFrame(dataset)
  for column in list(dataset.columns[dataset.isnull().sum() > 0]):
    mean_val = dataset[column].mean()
    dataset[column].fillna(mean_val, inplace=True)
  dataset = np.asarray(dataset).astype('float32')
  return dataset

def evaluation(predictions, y):                   # evaluate r2
  sse = np.sum((y - predictions) ** 2)
  ssr = np.sum((y - np.mean(y)) ** 2)

  sst = ssr + sse
  return 1 - sse / sst

def validate(predictions, y_validation):  # validation step
    #predictions = model(x_validation)
    loss = loss_object(y_validation, predictions)
    r2 = evaluation(predictions, y_validation)
    template = 'validate set: Loss {}, Evaluation {}'
    print(template.format(loss, r2))
    loss = K.eval(loss)
    return loss, r2

In [ ]:
data_dropout = random_remove(data, 0.5)

In [ ]:
data_dropout = pd.DataFrame(data_dropout)

In [ ]:
data_filled = fill_mean(data) #to fill remaining missing values in original dataset
data_scaled = (data_filled-data_filled.min())/(data_filled.max()-data_filled.min())

In [ ]:
data_filled = fill_mean(data) #to fill remaining missing values in original dataset
data_scaled = (data_filled-data_filled.min())/(data_filled.max()-data_filled.min())
data_scaled_rm = random_remove(data_scaled, 0.5)
data_scaled_pred = fill_mean(data_scaled_rm)

loss_object = keras.losses.MeanSquaredError()
l, r = validate(data_scaled_pred, data_scaled)
l

validate set: Loss 2.883957677113358e-06, Evaluation 0.9673461802303791


2.8839577e-06

In [ ]:
#imputer = md.Midas(layer_structure = [256,256], vae_layer = False, seed = 89, input_drop = 1)
#imputer.build_model(data_dropout)
#imputer.train_model(training_epochs = 25)

In [ ]:
#data_imputed = imputer.generate_samples(m=1).output_list

In [ ]:
#data_imputed[0].head()

In [ ]:
#loss_object = keras.losses.MeanSquaredError()

#loss, r2 = validate(data_imputed[0], fill_mean(data))

# **Try model**

In [ ]:
imputer_2 = md.Midas(layer_structure = [128,256, 128], vae_layer = False, seed = 89, input_drop = 1)
imputer_2.build_model(data_dropout)
imputer_2.train_model(training_epochs = 5)

data_imputed_2 = imputer_2.generate_samples(m=1).output_list
loss_object = keras.losses.MeanSquaredError()
loss_2, r2_2 = validate(data_imputed_2[0], fill_mean(data))

Size index: [67]

Computation graph constructed

Model initialised

Epoch: 0 , loss: 4721.01295214443
Epoch: 1 , loss: 4574.16829237827
Epoch: 2 , loss: 3829.739925107943
Epoch: 3 , loss: 2441.078517178015
Epoch: 4 , loss: 2090.6815633082356
Training complete. Saving file...
Model saved in file: tmp/MIDAS
INFO:tensorflow:Restoring parameters from tmp/MIDAS
Model restored.
validate set: Loss Tensor("mean_squared_error/weighted_loss/value:0", shape=(), dtype=float32), Evaluation 0     1.000000
1     0.999951
2     0.999896
3     0.997669
4     0.975126
        ...   
62    1.000000
63    0.999994
64    0.999994
65    1.000000
66    1.000000
Length: 67, dtype: float32


In [ ]:
print(loss_2, np.mean(r2_2))

904867.9 0.9842497706413269


# Test some hidden layer variations

In [ ]:
losses, r2s = [],[]
layers = [[20],[40],[60],[20,20], [40,40], [60,60], [20,20,20], [40,40,40], [60, 60, 60]]
for layer in layers:
  imputer_ = md.Midas(layer_structure = layer, vae_layer = False, seed = 89, input_drop = 1)
  imputer_.build_model(data_dropout)
  imputer_.train_model(training_epochs = 5)

  data_imputed_ = imputer_.generate_samples(m=1).output_list
  data_imputed_scaled =  (data_imputed_[0]-data_imputed_[0].min())/(data_imputed_[0].max()-data_imputed_[0].min())
  loss_object = keras.losses.MeanSquaredError()
  loss_, r2_ = validate(data_imputed_scaled, data_scaled)
  #print(layers, loss_, np.mean(r2_))
  losses.append(loss_)
  r2s.append(np.mean(r2_))
 

Size index: [67]

Computation graph constructed

Model initialised

Epoch: 0 , loss: 6439.943856673756
Epoch: 1 , loss: 4889.308615792793
Epoch: 2 , loss: 4530.832459080561
Epoch: 3 , loss: 4310.357063846614
Epoch: 4 , loss: 4061.2419259923263
Training complete. Saving file...
Model saved in file: tmp/MIDAS
INFO:tensorflow:Restoring parameters from tmp/MIDAS
Model restored.
validate set: Loss Tensor("mean_squared_error/weighted_loss/value:0", shape=(), dtype=float32), Evaluation 0     0.015318
1     0.254928
2     0.495381
3     0.066562
4     0.317892
        ...   
62    0.011638
63    0.193567
64    0.132717
65    0.115817
66    0.013402
Length: 67, dtype: float32
Size index: [67]

Computation graph constructed

Model initialised

Epoch: 0 , loss: 6930.739499949234
Epoch: 1 , loss: 4969.322590589197
Epoch: 2 , loss: 4396.753831438098
Epoch: 3 , loss: 3926.2041800482
Epoch: 4 , loss: 3498.4079526387227
Training complete. Saving file...
Model saved in file: tmp/MIDAS
INFO:tensorflow:R

In [ ]:
results = {'RMSE': losses, 'r2': r2s}
pd.DataFrame.from_dict(results, orient='index',
                       columns=[str(layer) for layer in layers])

,[20],[40],[60],"[20, 20]","[40, 40]","[60, 60]","[20, 20, 20]","[40, 40, 40]","[60, 60, 60]"
RMSE,0.147657,0.142597,0.165489,0.122525,0.089180,0.083970,0.128875,0.093013,0.084482
r2,0.210746,0.210206,0.116486,0.198237,0.218367,0.146647,0.177969,0.142649,0.182333


cursieve tekst

In [ ]:
losses, r2s = [],[]
layers = [[60,60], [70,70], [80,80]]
for layer in layers:
  imputer_ = md.Midas(layer_structure = layer, vae_layer = False, seed = 89, input_drop = 1)
  imputer_.build_model(data_dropout, verbose=False)
  imputer_.train_model(training_epochs = 5, verbose=False)

  data_imputed_ = imputer_.generate_samples(m=1).output_list
  data_imputed_scaled =  (data_imputed_[0]-data_imputed_[0].min())/(data_imputed_[0].max()-data_imputed_[0].min())
  loss_object = keras.losses.MeanSquaredError()
  loss_, r2_ = validate(data_imputed_scaled, data_scaled)
  #print(layers, loss_, np.mean(r2_))
  losses.append(loss_)
  r2s.append(np.mean(r2_))

results = {'RMSE': losses, 'r2': r2s}
pd.DataFrame.from_dict(results, orient='index',
                       columns=[str(layer) for layer in layers])

INFO:tensorflow:Restoring parameters from tmp/MIDAS
Model restored.
validate set: Loss Tensor("mean_squared_error/weighted_loss/value:0", shape=(), dtype=float32), Evaluation 0     0.302763
1     0.050821
2     0.089530
3     0.332879
4     0.053000
        ...   
62    0.049490
63    0.294081
64    0.082779
65    0.051717
66    0.028294
Length: 67, dtype: float32
INFO:tensorflow:Restoring parameters from tmp/MIDAS
Model restored.
validate set: Loss Tensor("mean_squared_error/weighted_loss/value:0", shape=(), dtype=float32), Evaluation 0     0.513261
1     0.083658
2     0.527147
3     0.317680
4     0.051843
        ...   
62    0.057493
63    0.407106
64    0.037044
65    0.588818
66    0.051659
Length: 67, dtype: float32
INFO:tensorflow:Restoring parameters from tmp/MIDAS
Model restored.
validate set: Loss Tensor("mean_squared_error/weighted_loss/value:0", shape=(), dtype=float32), Evaluation 0     0.101240
1     0.072215
2     0.102381
3     0.237076
4     0.050938
        ...   
6

,"[60, 60]","[70, 70]","[80, 80]"
RMSE,0.083575,0.081700,0.086885
r2,0.147898,0.194781,0.163783


In [ ]:
losses, r2s = [],[]
layers = [ [70,70, 70]]
for layer in layers:
  imputer_ = md.Midas(layer_structure = layer, vae_layer = False, seed = 89, input_drop = 1)
  imputer_.build_model(data_dropout, verbose=False)
  imputer_.train_model(training_epochs = 5, verbose=False)

  data_imputed_ = imputer_.generate_samples(m=1).output_list
  data_imputed_scaled =  (data_imputed_[0]-data_imputed_[0].min())/(data_imputed_[0].max()-data_imputed_[0].min())
  loss_object = keras.losses.MeanSquaredError()
  loss_, r2_ = validate(data_imputed_scaled, data_scaled)
  #print(layers, loss_, np.mean(r2_))
  losses.append(loss_)
  r2s.append(np.mean(r2_))

results = {'RMSE': losses, 'r2': r2s}
pd.DataFrame.from_dict(results, orient='index',
                       columns=[str(layer) for layer in layers])

INFO:tensorflow:Restoring parameters from tmp/MIDAS
Model restored.
validate set: Loss Tensor("mean_squared_error/weighted_loss/value:0", shape=(), dtype=float32), Evaluation 0     0.079099
1     0.123054
2     0.437835
3     0.208910
4     0.052670
        ...   
62    0.024998
63    0.375872
64    0.115419
65    0.051725
66    0.059652
Length: 67, dtype: float32


,"[70, 70, 70]"
RMSE,0.087088
r2,0.176322


# **[70, 70] is best hidden layer setting**

In [ ]:
losses, r2s = [],[]
layers = [ [70,70]]
for layer in layers:
  imputer_ = md.Midas(layer_structure = layer, vae_layer = False, seed = 89, input_drop = 1)
  imputer_.build_model(data_dropout, verbose=False)
  imputer_.train_model(training_epochs = 25, verbose=False)

  data_imputed_ = imputer_.generate_samples(m=1).output_list
  data_imputed_scaled =  (data_imputed_[0]-data_imputed_[0].min())/(data_imputed_[0].max()-data_imputed_[0].min())
  loss_object = keras.losses.MeanSquaredError()
  loss_, r2_ = validate(data_imputed_scaled, data_scaled)
  #print(layers, loss_, np.mean(r2_))
  losses.append(loss_)
  r2s.append(np.mean(r2_))

results = {'RMSE': losses, 'r2': r2s}
pd.DataFrame.from_dict(results, orient='index',
                       columns=[str(layer) for layer in layers])

INFO:tensorflow:Restoring parameters from tmp/MIDAS
Model restored.
validate set: Loss Tensor("mean_squared_error/weighted_loss/value:0", shape=(), dtype=float32), Evaluation 0     0.115717
1     0.028079
2     0.040178
3     0.154787
4     0.049717
        ...   
62    0.116125
63    0.051562
64    0.136950
65    0.028191
66    0.027722
Length: 67, dtype: float32


,"[70, 70]"
RMSE,0.118882
r2,0.115743
